1
Обучите простую рекуррентную нейронную сеть (без GRU/LSTM, без внимания) решать задачу дешифровки шифра Цезаря:
Написать алгоритм шифра Цезаря для генерации выборки (сдвиг на N каждой буквы). Например если N=2, то буква A переходит в букву C. Можно поиграться с языком на выбор (немецкий, русский и т.д.)
Создать архитектуру рекуррентной нейронной сети.
Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза).
Проверить качество модели.	
2 балла за правильно выполненное задание.


In [7]:
#преднастройка
import torch
import torch.nn as nn
import torch.optim as optim
import random
import string

In [40]:
# параметры
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ALPHABET = "АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя0123456789 "+string.punctuation #"абвгдеёжзийклмнопрстуфхцчшщъыьэюя"
CHAR_TO_INDEX = {ch: i for i, ch in enumerate(ALPHABET)}
INDEX_TO_CHAR = {i: ch for i, ch in enumerate(ALPHABET)}
SEQ_LENGTH = 100  # длина последовательности
HIDDEN_SIZE = 128
EPOCHS = 2
BATCH_SIZE = 64
SHIFT = 7  # cдвиг шифра Цезаря

In [42]:
# вспомогательные функции
# функция шифрования текста
def caesar_encrypt(text, shift):
    return "".join(ALPHABET[(CHAR_TO_INDEX[ch] + shift) % len(ALPHABET)] for ch in text)


# функция расшифровки
def caesar_decrypt(text, shift):
    return "".join(ALPHABET[(CHAR_TO_INDEX[ch] - shift) % len(ALPHABET)] for ch in text)


# генерация обучающих данных
def generate_data(num_samples):
    data = []
    for _ in range(num_samples):
        x = "".join(random.choices(ALPHABET, k=SEQ_LENGTH))
        y = caesar_decrypt(x, SHIFT)
        data.append((x, y))
    return data


# преобразование строки в тензор
def text_to_tensor(text):
    return torch.tensor([CHAR_TO_INDEX[ch] for ch in text], dtype=torch.long, device=DEVICE)


# преобразование тензора в строку
def tensor_to_text(tensor):
    return "".join(INDEX_TO_CHAR[idx] for idx in tensor.tolist())


In [44]:
# определение RNN-модели
class CaesarRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CaesarRNN, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        rnn_out, _ = self.rnn(embedded)
        output = self.fc(rnn_out)
        return output


In [46]:
# cоздание модели
model = CaesarRNN(len(ALPHABET), HIDDEN_SIZE, len(ALPHABET)).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# подготовка данных
train_data = generate_data(1000)
x_train = torch.stack([text_to_tensor(x) for x, _ in train_data])
y_train = torch.stack([text_to_tensor(y) for _, y in train_data])


In [48]:
# обучения
def train():
    for epoch in range(EPOCHS):
        total_loss = 0
        for i in range(0, len(x_train), BATCH_SIZE):
            x_batch = x_train[i:i + BATCH_SIZE]
            y_batch = y_train[i:i + BATCH_SIZE]

            optimizer.zero_grad()
            output = model(x_batch)
            loss = criterion(output.view(-1, len(ALPHABET)), y_batch.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {total_loss / len(x_train):.8f}")


train()

# сохранение модели
torch.save(model.state_dict(), "caesar_model.pth")

In [49]:
# функция расшифровки
def decrypt_text(encrypted_text):
    model.eval()
    x_test = text_to_tensor(encrypted_text).unsqueeze(0)
    with torch.no_grad():
        output = model(x_test)
        predicted_indices = output.argmax(dim=2).squeeze()
    return tensor_to_text(predicted_indices)


# тестирование модели
test_text = "Привет Мир! & наша группа #2025 года выпуска"
encrypted_text = caesar_encrypt(test_text, SHIFT)
decrypted_text = decrypt_text(encrypted_text)

print(f"Исходный текст: {test_text}")
print(f"Зашифрованный текст: {encrypted_text}")
print(f"Расшифрованный текст: {decrypted_text}")


# тестирование модели на сгенерированных данных
num_test_samples = 10000
test_data = generate_data(num_test_samples)
correct_count = 0

for enc_text, true_dec_text in test_data:
    pred_dec_text = decrypt_text(enc_text)
    if pred_dec_text == true_dec_text:
        correct_count += 1

accuracy = (correct_count / num_test_samples) * 100
print(f"Точность расшифровки на {num_test_samples} тестовых примеров: {accuracy:.2f}%")

Исходный текст: Привет Мир! & наша группа #2025 года выпуска
Зашифрованный текст: Цчпилщ'Упч('-'фжяж'йчъццж'*979"'йхкж'и2цъшсж
Расшифрованный текст: Привет Мир! & наша группа #2025 года выпуска
Точность расшифровки на 10000 тестовых примеров: 100.00%


In [62]:
#залитая моделька с эндпоинтом
import requests

url = "http://45.152.115.10:5000/decrypt_caesar"
data = {"encrypted_text": "Цчпилщ'Упч('-'фжяж'йчъццж'*979\"'йхкж'и2цъшсж"}
response = requests.post(url, json=data)

print(response.json())

{'decrypted_text': 'Привет Мир! & наша группа #2025 года выпуска'}


2. 	
Сгенерировать последовательности, которые состоят из цифр (от 0 до 9) и задаются следующим образом:
x - последовательность цифр
y1 = x1 
yi = xi + x1
Если yi >= 10 то yi = yi - 10
Научить модель рекуррентной нейронной сети предсказывать yi по xi Использовать: RNN, LSTM, GRU
6 баллов за правильно выполненное задание.


In [64]:
#преднастройка
import torch
import torch.nn as nn
import torch.optim as optim
import random

In [66]:
# параметры
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DIGITS = list(range(10))  # Цифры от 0 до 9
SEQ_LENGTH = 10  # Длина входной последовательности
HIDDEN_SIZE = 128
EPOCHS = 500
BATCH_SIZE = 64

In [107]:
#разное
def compute_y(x):
    y = []
    x1 = x[0]  
    for i, xi in enumerate(x):
        if i == 0:
            yi = x1  
        else:
            yi = xi + x1
            if yi >= 10:
                yi -= 10  
        y.append(yi)
    return y


# генерация выборки
def generate_data(num_samples):
    data = []
    for _ in range(num_samples):
        x = [random.choice(DIGITS) for _ in range(SEQ_LENGTH)]
        y = compute_y(x)
        data.append((x, y))
    return data


# в тензор
def sequence_to_tensor(seq):
    return torch.tensor(seq, dtype=torch.long, device=DEVICE)


def tensor_to_sequence(tensor):
    return tensor.tolist()

In [109]:
# архитектура RNN LSTM GRU
class SequenceRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, rnn_type="RNN"):
        super(SequenceRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)

        if rnn_type == "LSTM":
            self.rnn = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        elif rnn_type == "GRU":
            self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True)
        else:
            self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)

        #s
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        rnn_out, _ = self.rnn(embedded)
        #s
        rnn_out = self.dropout(rnn_out)
        output = self.fc(rnn_out)
        return output


In [111]:
# создание модели RNN
rnn_type = "RNN"  # RNN", "LSTM" или "GRU"
model = SequenceRNN(len(DIGITS), HIDDEN_SIZE, len(DIGITS), rnn_type).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001) #0.01

# подготовка данных
train_data = generate_data(30000)
x_train = torch.stack([sequence_to_tensor(x) for x, _ in train_data])
y_train = torch.stack([sequence_to_tensor(y) for _, y in train_data])


In [112]:
# тренировка RNN

def train():
    for epoch in range(EPOCHS):
        total_loss = 0
        for i in range(0, len(x_train), BATCH_SIZE):
            x_batch = x_train[i:i + BATCH_SIZE]
            y_batch = y_train[i:i + BATCH_SIZE]

            optimizer.zero_grad()
            output = model(x_batch)
            loss = criterion(output.view(-1, len(DIGITS)), y_batch.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {total_loss / len(x_train):.8f}")


train()
# сохранение модели
torch.save(model.state_dict(), "gener_rnn.pth")

Epoch 50/500, Loss: 0.00003064
Epoch 100/500, Loss: 0.00000757
Epoch 150/500, Loss: 0.00000412
Epoch 200/500, Loss: 0.00000323
Epoch 250/500, Loss: 0.00000247
Epoch 300/500, Loss: 0.00000180
Epoch 350/500, Loss: 0.00000134
Epoch 400/500, Loss: 0.00000129
Epoch 450/500, Loss: 0.00000152
Epoch 500/500, Loss: 0.00000098


In [114]:
# тестирование RNN
model.eval()

incorrect_predictions = 0
total_predictions = 0

for i in range(10000):
    test_sequence = [random.choice(DIGITS) for _ in range(random.randint(SEQ_LENGTH, SEQ_LENGTH * 2))]
    y_expected = compute_y(test_sequence)
    x_test = sequence_to_tensor(test_sequence).unsqueeze(0)
    output = model(x_test)
    predicted_indices = output.argmax(dim=2).squeeze().tolist()
    if y_expected != predicted_indices:
        incorrect_predictions += 1
    total_predictions += 1
    if i==0:
        print(f"test_sequence:{test_sequence} y_expected:{y_expected} predicted_indices:{predicted_indices}")

# Вычисление процента ошибок
error_percentage = (incorrect_predictions / total_predictions) * 100
print(f"Процент неверных предсказаний RNN: {error_percentage:.2f}%")

test_sequence:[1, 4, 2, 6, 5, 8, 2, 7, 6, 2, 3, 5, 9, 5, 5, 9] y_expected:[1, 5, 3, 7, 6, 9, 3, 8, 7, 3, 4, 6, 0, 6, 6, 0] predicted_indices:[1, 5, 3, 7, 6, 9, 3, 8, 7, 3, 4, 6, 0, 6, 6, 0]
Процент неверных предсказаний RNN: 0.00%


In [115]:
# создание модели LSTM
rnn_type = "LSTM"  # RNN", "LSTM" или "GRU"
model = SequenceRNN(len(DIGITS), HIDDEN_SIZE, len(DIGITS), rnn_type).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001) #0.01

# подготовка данных
train_data = generate_data(30000)
x_train = torch.stack([sequence_to_tensor(x) for x, _ in train_data])
y_train = torch.stack([sequence_to_tensor(y) for _, y in train_data])

In [116]:
# тренировка LSTM

def train():
    for epoch in range(EPOCHS):
        total_loss = 0
        for i in range(0, len(x_train), BATCH_SIZE):
            x_batch = x_train[i:i + BATCH_SIZE]
            y_batch = y_train[i:i + BATCH_SIZE]

            optimizer.zero_grad()
            output = model(x_batch)
            loss = criterion(output.view(-1, len(DIGITS)), y_batch.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {total_loss / len(x_train):.8f}")


train()
# сохранение модели
torch.save(model.state_dict(), "gener_lstm.pth")

Epoch 50/500, Loss: 0.00000002
Epoch 100/500, Loss: 0.00000000
Epoch 150/500, Loss: 0.00000000
Epoch 200/500, Loss: 0.00000000
Epoch 250/500, Loss: 0.00000000
Epoch 300/500, Loss: 0.00000000
Epoch 350/500, Loss: 0.00000000
Epoch 400/500, Loss: 0.00000000
Epoch 450/500, Loss: 0.00000000
Epoch 500/500, Loss: 0.00000000


In [117]:
# тестирование RNN
model.eval()

incorrect_predictions = 0
total_predictions = 0

for i in range(10000):
    test_sequence = [random.choice(DIGITS) for _ in range(random.randint(SEQ_LENGTH, SEQ_LENGTH * 2))]
    y_expected = compute_y(test_sequence)
    x_test = sequence_to_tensor(test_sequence).unsqueeze(0)
    output = model(x_test)
    predicted_indices = output.argmax(dim=2).squeeze().tolist()
    if y_expected != predicted_indices:
        incorrect_predictions += 1
    total_predictions += 1
    if i==0:
        print(f"test_sequence:{test_sequence} y_expected:{y_expected} predicted_indices:{predicted_indices}")

# Вычисление процента ошибок
error_percentage = (incorrect_predictions / total_predictions) * 100
print(f"Процент неверных предсказаний LSTM: {error_percentage:.2f}%")

test_sequence:[8, 3, 5, 8, 3, 7, 9, 5, 1, 7, 7] y_expected:[8, 1, 3, 6, 1, 5, 7, 3, 9, 5, 5] predicted_indices:[8, 1, 3, 6, 1, 5, 7, 3, 9, 5, 5]
Процент неверных предсказаний LSTM: 0.00%


In [118]:
# создание модели GRU
rnn_type = "GRU"  # RNN", "LSTM" или "GRU"
model = SequenceRNN(len(DIGITS), HIDDEN_SIZE, len(DIGITS), rnn_type).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001) #0.01

# подготовка данных
train_data = generate_data(30000)
x_train = torch.stack([sequence_to_tensor(x) for x, _ in train_data])
y_train = torch.stack([sequence_to_tensor(y) for _, y in train_data])

In [119]:
# тренировка RNN

def train():
    for epoch in range(EPOCHS):
        total_loss = 0
        for i in range(0, len(x_train), BATCH_SIZE):
            x_batch = x_train[i:i + BATCH_SIZE]
            y_batch = y_train[i:i + BATCH_SIZE]

            optimizer.zero_grad()
            output = model(x_batch)
            loss = criterion(output.view(-1, len(DIGITS)), y_batch.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {total_loss / len(x_train):.8f}")


train()
# сохранение модели
torch.save(model.state_dict(), "gener_gru.pth")

Epoch 50/500, Loss: 0.00000037
Epoch 100/500, Loss: 0.00000004
Epoch 150/500, Loss: 0.00000001
Epoch 200/500, Loss: 0.00000000
Epoch 250/500, Loss: 0.00000001
Epoch 300/500, Loss: 0.00000000
Epoch 350/500, Loss: 0.00000000
Epoch 400/500, Loss: 0.00000000
Epoch 450/500, Loss: 0.00000000
Epoch 500/500, Loss: 0.00000000


In [120]:
# тестирование GRU
model.eval()

incorrect_predictions = 0
total_predictions = 0

for i in range(10000):
    test_sequence = [random.choice(DIGITS) for _ in range(random.randint(SEQ_LENGTH, SEQ_LENGTH * 2))]
    y_expected = compute_y(test_sequence)
    x_test = sequence_to_tensor(test_sequence).unsqueeze(0)
    output = model(x_test)
    predicted_indices = output.argmax(dim=2).squeeze().tolist()
    if y_expected != predicted_indices:
        incorrect_predictions += 1
    total_predictions += 1
    if i==0:
        print(f"test_sequence:{test_sequence} y_expected:{y_expected} predicted_indices:{predicted_indices}")

# Вычисление процента ошибок
error_percentage = (incorrect_predictions / total_predictions) * 100
print(f"Процент неверных предсказаний GRU: {error_percentage:.2f}%")

test_sequence:[9, 3, 4, 7, 6, 7, 1, 6, 2, 5, 4, 8, 4, 1, 7, 1, 4] y_expected:[9, 2, 3, 6, 5, 6, 0, 5, 1, 4, 3, 7, 3, 0, 6, 0, 3] predicted_indices:[9, 2, 3, 6, 5, 6, 0, 5, 1, 4, 3, 7, 3, 0, 6, 0, 3]
Процент неверных предсказаний GRU: 0.00%


In [131]:
#залитая моделька с эндпоинтом
import requests

url = "http://45.152.115.10:5000/predict_sequence"
data = {"sequence": [9, 3, 4, 7, 6, 7, 1, 6, 2, 5, 4, 8, 4, 1, 7, 1, 4]}
response = requests.post(url, json=data)

print(response.json())

{'predicted': [9, 2, 3, 6, 5, 6, 0, 5, 1, 4, 3, 7, 3, 0, 6, 0, 3]}


3. 
Решить задачу машинного перевода выбрав свой язык:
Формируем датасет с исходного языка на целевой (код прописать в классе)
Строим архитектуру нейронной сети 
Обучаем 
Проверить качество с помощью метрики BLEU
2 балла за правильно выполненное задание.

In [ ]:
в другом блокноте